In [1]:
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt

# 1. Initialize MC

In [137]:
def init_chain():
    
    # initialize Q with only zeros
    Q = np.zeros((5,2))
    
    # for each state, action 1 and action 2
    transitions = {0: [1, 0], 
                   1: [2, 0], 
                   2: [3, 0], 
                   3: [4, 0],
                   4: [4, 0]}
    
    # rewards for each state's actions
    R = np.array([[0, 0.2],
                  [0, 0  ],
                  [0, 0  ],
                  [0, 0  ],
                  [1, 0  ]])
    
    gamma = 0.9
    
    return Q, transitions, R, gamma

# 2. Tabular Q-learning

In [138]:
def BK(state, old_Q, action, next_state, R, gamma):

    # tabular values of 
    A = old_Q[next_state, :]
    
    # immediate reward of the current state, for either action 1 or action 2
    R_xa = R[state][action]

    return R_xa + gamma * max(A)
    

def tabular_Qlearning(Q, transitions, R, gamma):
    
    # loop until converged
    converged = False
    while not converged:
        
        # save copy of the Q in the previous timestep
        old_Q = deepcopy(Q)

        # iterate over all states
        for state in range(len(Q)):

            # retrieve all possible next states
            next_states = np.array(transitions[state])

            #calculate BK(state, action) for action 1 and action 2
            a1, a2 = [BK(state, old_Q, action, next_state, R, gamma) 
                      for (action, next_state) in enumerate(next_states)]
            
            # update Q with BK(state, action )
            Q[state] = a1, a2
            
        # stop if the Q has converged
        if np.array_equal(Q, old_Q):
            converged = True
            
    return Q

# initialize Q and the corresponding parameters
Q, transitions, R, gamma = init_chain()

Q = tabular_Qlearning(Q, transitions, R, gamma)
print(Q)

[[ 6.561   6.1049]
 [ 7.29    5.9049]
 [ 8.1     5.9049]
 [ 9.      5.9049]
 [10.      5.9049]]


# 3. Epsilon greedy

In [2]:
def init_Q():
    return np.zeros((5,2))

def init_chain():
    
    # for each state, action 1 and action 2
    transitions = {0: [1, 0], 
                   1: [2, 0], 
                   2: [3, 0], 
                   3: [4, 0],
                   4: [4, 0]}
    
    # rewards for each state's actions
    R = np.array([[0, 0.2],
                  [0, 0  ],
                  [0, 0  ],
                  [0, 0  ],
                  [1, 0  ]])
    
    return transitions, R

In [3]:
def update_Q(Q, state, action, next_state, R, gamma, alpha, epsilon):
    
    Q_xa = Q[state][action]
    
    # tabular values of next state
    Q_ya = Q[next_state, :]
    
    # immediate reward of the current state, for either action 1 or action 2
    R_xa = R[state][action]

    return Q_xa + alpha * (R_xa + gamma * max(Q_ya) - Q_xa)

def Qlearning(Q, transitions, R, gamma, alpha, epsilon):
    
    for episode in range(100):
        
        # initialize starting state
        state = 0
        
        for step in range(200):

            # retrieve all possible next states
            next_states = np.array(transitions[state])

            # greedy action
            if np.random.sample() < (1-epsilon):
                action = np.argmax(Q[state])
                next_state = next_states[action]
                
            # random action
            else:
                action = np.random.choice([0,1])
                next_state = next_states[action]

            Q[state][action] = update_Q(Q, state, action, next_state, R, gamma, alpha, epsilon)
            
            state = next_state
    
    return Q

# initialize Q and the chain
Q = init_Q()
transitions, R = init_chain()

# initialize learing paramters
gamma = 0.9
alpha = 0.99
epsilon = 0.1

Q = Qlearning(Q, transitions, R, gamma, alpha, epsilon)
Q

array([[ 6.561 ,  6.1049],
       [ 7.29  ,  5.9049],
       [ 8.1   ,  5.9049],
       [ 9.    ,  5.9049],
       [10.    ,  5.9049]])